In [11]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Fix random seed
np.random.seed(1)


In [18]:
!pip install biosppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.2/142.2 kB 3.1 MB/s eta 0:00:00


In [8]:
# Go to google drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%cd /content/drive/MyDrive/AML/Task2

/content/drive/MyDrive/AML/Task2


**Install BioSPPy**

**Import Data**

In [17]:
df_x_train = pd.read_csv('X_train.csv')
df_y_train = pd.read_csv('y_train.csv')
df_x_test = pd.read_csv('X_test.csv')
df_sample = pd.read_csv('sample.csv')

# drop the column id
df_x_train = df_x_train.drop('id', axis=1)
df_y_train = df_y_train.drop('id', axis=1)
df_x_test = df_x_test.drop('id', axis=1)



In [19]:
# to numpy
x_train_total = df_x_train.to_numpy(dtype='float32')
y_train_total = df_y_train.to_numpy()
x_test = df_x_test.to_numpy(dtype='float32')

**Feature extraction**

In [20]:
import biosppy.signals.ecg as ecg

# hyper parameter:
th = 0.48 #r-peak segmentation

def extract_features(X):
  X_features = np.array([])
  n_features = 34
  for row in X:
    # row without Nans
    row = row[np.logical_not(np.isnan(row))]
    # R-peak location indices
    r_peaks = ecg.engzee_segmenter(row, sampling_rate=300, threshold=th)['rpeaks']
    if r_peaks.shape == (0,): #TODO: what to do with those samples?
      features = np.zeros(n_features) #set all features to zero if no R-peak was found
      print("No R-peak found in sample: ", X_features.shape[0])
    else:
      # heartbeat templates
      beats = ecg.extract_heartbeats(row, r_peaks, sampling_rate=300, before=0.2, after=0.4)['templates']
      beats_var_mean = np.mean(np.std(beats, axis=0))
      beats_var_md = np.median(np.std(beats, axis=0))
      beats_var_var = np.std(np.std(beats, axis=0))

      amplitude_mean = np.mean(np.mean(np.power(beats, 2), axis=1)) #squared
      amplitude_md = np.median(np.mean(np.power(beats, 2), axis=1)) #squared
      amplitude_var = np.std(np.mean(np.power(beats, 2), axis=1)) #squared

      # R-peak: always at T=60
      r_mean = np.mean(beats[:, 60])
      r_md = np.median(beats[:, 60])
      r_var = np.std(beats[:, 60])

      # P-peak: smaller peak before R-peak, 40 good?
      p_mean = np.mean(np.amax(beats[:, :40], axis=1))
      p_md = np.median(np.amax(beats[:, :40], axis=1))
      p_var = np.std(np.amax(beats[:, :40], axis=1))
      p_position = np.mean(np.argmax(beats[:, :40], axis=1))
      p_position_var = np.std(np.argmax(beats[:, :40], axis=1))
      if p_position == 40.0:
        print("P peak earlier") #TODO: position good?

      # Q-drop: before R-peak
      q_mean = np.mean(np.amin(beats[:, :60], axis=1))
      q_md = np.median(np.amin(beats[:, :60], axis=1))
      q_var = np.std(np.amin(beats[:, :60], axis=1))
      q_position = np.mean(np.argmin(beats[:, :60], axis=1))
      q_position_var = np.std(np.argmin(beats[:, :60], axis=1))

      # S-drop: after R-peak
      s_mean = np.mean(np.amin(beats[:, 60:], axis=1))
      s_md = np.median(np.amin(beats[:, 60:], axis=1))
      s_var = np.std(np.amin(beats[:, 60:], axis=1))
      s_position = np.mean(np.argmin(beats[:, 60:], axis=1))
      s_position_var = np.std(np.argmin(beats[:, 60:], axis=1))

      # T-peak: after R-peak, 80 good?
      t_mean = np.mean(np.amax(beats[:, 80:], axis=1))
      t_md = np.median(np.amax(beats[:, 80:], axis=1))
      t_var = np.std(np.amax(beats[:, 80:], axis=1))
      t_position = np.mean(np.argmax(beats[:, 80:], axis=1))
      t_position_var = np.std(np.argmax(beats[:, 80:], axis=1))
      if t_position == 0.0:
        print("T peak later") #TODO: position good?

      # instantaneous heart rate (bpm)
      heart_rate = ecg.ecg(row, 300, show=False)['heart_rate']
      if heart_rate.shape == (0,):
        hr_mean = 0
        hr_md = 0
        hr_var = 0
        hr_max = 0
        hr_min = 0
      else:
        hr_mean = np.mean(heart_rate)
        hr_md = np.median(heart_rate)
        hr_var = np.std(heart_rate)
        hr_max = np.amax(heart_rate)
        hr_min = np.amin(heart_rate)

      features = np.array([beats_var_mean, beats_var_md, beats_var_var,
                  amplitude_mean, amplitude_md, amplitude_var,
                  r_mean, r_md, r_var,
                  p_mean, p_md, p_var, p_position, p_position_var,
                  q_mean, q_md, q_var, q_position, q_position_var,
                  s_mean, s_md, s_var, s_position, s_position_var,
                  t_mean, t_md, t_var, t_position, t_position_var,
                  hr_mean, hr_md, hr_var, hr_max, hr_min])

    if X_features.shape == (0,):
      X_features = features
    else:
      X_features = np.vstack((X_features, features))

  # Normalize
  X_features = np.array(X_features)
  m = np.nanmean(X_features, axis=0)
  s = np.nanstd(X_features, axis=0)
  X_features = X_features - m
  X_features = X_features / s

  return X_features

In [21]:
x_train_total_features = extract_features(x_train_total)

T peak later
T peak later
No R-peak found in sample:  447
No R-peak found in sample:  579
No R-peak found in sample:  593
No R-peak found in sample:  955
No R-peak found in sample:  1135
T peak later
No R-peak found in sample:  1247
No R-peak found in sample:  1383
No R-peak found in sample:  2931
No R-peak found in sample:  3009
T peak later
No R-peak found in sample:  3145
T peak later
T peak later
No R-peak found in sample:  3519
T peak later
No R-peak found in sample:  3884
No R-peak found in sample:  4025
T peak later
T peak later
No R-peak found in sample:  4340
T peak later
No R-peak found in sample:  4625
No R-peak found in sample:  4684
No R-peak found in sample:  4688
No R-peak found in sample:  4693
No R-peak found in sample:  4871


# **Cross-Validate**
## Instead of model one can use any classifier. Moreover, the best can be stacked many xgboost(more than 10), random forest, and ridge regression as the final classifier.

In [22]:
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn import svm
from sklearn.metrics import f1_score

s=0
n_splits = 10
val_score_sum = 0
kf = KFold(n_splits=n_splits)
for train_index, test_index in kf.split(x_train_total_features):
  x_train, x_val = x_train_total_features[train_index], x_train_total_features[test_index]
  y_train, y_val = y_train_total[train_index], y_train_total[test_index]
  s += 1
  print("Split ", s, " of ", n_splits)


  # Class Imbalance

  train = np.concatenate((y_train, x_train), axis=1)
  y_train = y_train.ravel() #reshape
  train_0 = train[y_train == 0]
  train_1 = train[y_train == 1]
  train_2 = train[y_train == 2]
  train_3 = train[y_train == 3]


  # upsample minority classes (Class 0 is the biggest class)
  train_1_upsampled = resample(train_1,
                            replace=True, # sample with replacement
                            n_samples=len(train_0)) # match number in majority class

  train_2_upsampled = resample(train_2,
                            replace=True, # sample with replacement
                            n_samples=len(train_0)) # match number in majority class

  train_3_upsampled = resample(train_3,
                            replace=True, # sample with replacement
                            n_samples=len(train_0)) # match number in majority class

  train_upsampled = np.concatenate((train_0, train_1_upsampled, train_2_upsampled, train_3_upsampled), axis=0)

  y_train_upsampled, x_train_upsampled = np.split(train_upsampled, [1], axis=1)

  y_train_upsampled = y_train_upsampled.ravel() #reshape

  # Classification Model

  model = svm.SVC(C=100.0, class_weight='balanced', probability=True, tol=1e-6)

  model.fit(x_train_upsampled, y_train_upsampled)


  # Evaluation

  #training set
  y_train_pred = model.predict(x_train_upsampled)
  F1 = f1_score(y_train_upsampled, y_train_pred, average='micro')
  print('F1 score of training set: ', F1)

  #validation set
  y_val_pred = model.predict(x_val)
  F1 = f1_score(y_val, y_val_pred, average='micro')
  print('F1 score of validation set: ', F1)
  val_score_sum += F1

val_score_mean = val_score_sum/n_splits
print("CV score: ", val_score_mean)

Split  1  of  10
F1 score of training set:  0.9488615497612927
F1 score of validation set:  0.732421875
Split  2  of  10
F1 score of training set:  0.9555209097578871
F1 score of validation set:  0.720703125
Split  3  of  10
F1 score of training set:  0.9496521420725009
F1 score of validation set:  0.71484375
Split  4  of  10
F1 score of training set:  0.9548969072164949
F1 score of validation set:  0.6953125
Split  5  of  10
F1 score of training set:  0.946107234667646
F1 score of validation set:  0.701171875
Split  6  of  10
F1 score of training set:  0.9474071349760942
F1 score of validation set:  0.728515625
Split  7  of  10
F1 score of training set:  0.9490353112486348
F1 score of validation set:  0.708984375
Split  8  of  10
F1 score of training set:  0.9456401766004415
F1 score of validation set:  0.7142857142857143
Split  9  of  10
F1 score of training set:  0.9503474762253109
F1 score of validation set:  0.6731898238747553
Split  10  of  10
F1 score of training set:  0.9495060

**Fit and predict on whole training set**

In [23]:
x_test_features = extract_features(x_test)

No R-peak found in sample:  155
No R-peak found in sample:  172
No R-peak found in sample:  450
No R-peak found in sample:  509
No R-peak found in sample:  646
No R-peak found in sample:  704
T peak later
No R-peak found in sample:  950
No R-peak found in sample:  970
No R-peak found in sample:  1040
No R-peak found in sample:  1147
T peak later
T peak later
No R-peak found in sample:  1740
No R-peak found in sample:  2223
No R-peak found in sample:  2267
No R-peak found in sample:  2304
No R-peak found in sample:  2501
No R-peak found in sample:  3169
No R-peak found in sample:  3381
T peak later


**Class Imbalance**

In [24]:
train = np.concatenate((y_train_total, x_train_total_features), axis=1)
y_train_total = y_train_total.reshape(y_train_total.shape[0])
train_0 = train[y_train_total == 0]
train_1 = train[y_train_total == 1]
train_2 = train[y_train_total == 2]
train_3 = train[y_train_total == 3]


# upsample minority classes (Class 0 is the biggest class)
train_1_upsampled = resample(train_1,
                          replace=True, # sample with replacement
                          n_samples=len(train_0)) # match number in majority class

train_2_upsampled = resample(train_2,
                          replace=True, # sample with replacement
                          n_samples=len(train_0)) # match number in majority class

train_3_upsampled = resample(train_3,
                          replace=True, # sample with replacement
                          n_samples=len(train_0)) # match number in majority class

train_upsampled = np.concatenate((train_0, train_1_upsampled, train_2_upsampled, train_3_upsampled), axis=0)

y_train_upsampled, x_train_upsampled = np.split(train_upsampled, [1], axis=1)


In [25]:
model.fit(x_train_upsampled, y_train_upsampled)
y_pred = model.predict(x_test_features)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
y_pred.shape

(3411,)

**Save to file**

In [33]:
df_sample = pd.DataFrame(y_pred,)
print(df_sample.head())
df_sample.to_csv('pred.csv', index=False)

     0
0  0.0
1  2.0
2  3.0
3  0.0
4  2.0
